# Models to ONNX

## Environment Set Up

### Requirements

In [1]:
!pip install monai[einops] onnx

### Colab Integration

In [2]:
# Set up Colab Workspace
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

!ln -s /content/drive/MyDrive/TFM/utils /content/utils
!ln -s /content/drive/MyDrive/TFM/outputs /content/outputs

Mounted at /content/drive
ln: failed to create symbolic link '/content/utils/utils': File exists
ln: failed to create symbolic link '/content/outputs/outputs': File exists


### Imports

In [3]:
import torch
from tqdm import tqdm
from utils.Models import SEGRESNET, UNET, AHNET, UNTR

### Cuda Config

In [4]:
# Check if CUDA is available
device = None
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on GPU")
else:
    device = torch.device("cpu")
    print("Running on CPU")

# Print the device
print(f"Device: {device}")

Running on GPU
Device: cuda


## Store Models

In [5]:
models = {
    'SegResNet': SEGRESNET,
    'UNet': UNET,
    'AHNet': AHNET,
    'UNETR': UNTR
}

for model_name, model in tqdm(models.items()):
  model.to(device)
  model.eval()
  if model_name == 'AHNet':
    torch.onnx.export(model, torch.randn(1, 4, 256, 256, 160).to(device), f'outputs/{model_name}/{model_name}.onnx', opset_version=12)
  else:
    torch.onnx.export(model, torch.randn(1, 4, 240, 240, 160).to(device), f'outputs/{model_name}/{model_name}.onnx', opset_version=12)
  torch.cuda.empty_cache()

 25%|██▌       | 1/4 [00:01<00:04,  1.53s/it]/usr/local/lib/python3.10/dist-packages/torch/onnx/symbolic_helper.py:1515: UserWarning: ONNX export mode is set to TrainingMode.EVAL, but operator 'instance_norm' is set to train=True. Exporting with train=True.
  warnings.warn(
 50%|█████     | 2/4 [00:02<00:02,  1.21s/it]/usr/local/lib/python3.10/dist-packages/monai/networks/nets/ahnet.py:80: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if out.size() != residual.size():
100%|██████████| 4/4 [00:14<00:00,  3.71s/it]
